<a href="https://colab.research.google.com/github/skimaza/assist_ai/blob/main/mlp_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝의 이해
# MLP Regression

# https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379


In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt

import numpy as np
import imageio
from IPython.display import Image

In [ ]:
torch.manual_seed(1)    # 실행할 때마다 같은 결과를 내기 위해 random seed를 고정

x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

figsize = (10,5)
# view data
plt.figure(figsize=figsize)
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis Training Data')
plt.xlabel('Independent varible (x)')
plt.ylabel('Dependent varible (y)')
plt.show()



In [ ]:
# this is one way to define a network
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


In [ ]:

net = Net(n_feature=1, n_hidden=10, n_output=1)     # define the network
# print(net)  # net architecture
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


In [ ]:

my_images = []
fig, ax = plt.subplots(figsize=figsize)

# train the network
for t in range(200):
    print('epoch', t)
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    # plot and show learning process
    plt.cla()
    ax.set_title('Regression Analysis', fontsize=24)
    ax.set_xlabel('Independent variable (x)', fontsize=18)
    ax.set_ylabel('Dependent variable (y)', fontsize=18)
    ax.set_xlim(-1.05, 1.5)
    ax.set_ylim(-0.25, 1.25)
    ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
    ax.plot(x.data.numpy(), prediction.data.numpy(), 'g-', lw=3)
    ax.text(1.0, 0.1, 'Step = %d' % t, fontdict={'size': 18, 'color':  'red'})
    ax.text(1.0, 0, 'Loss = %.4f' % loss.data.numpy(),
            fontdict={'size': 18, 'color':  'red'})

    # Used to return the plot as an image array 
    # (https://ndres.me/post/matplotlib-animated-gifs-easily/)
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    my_images.append(image)
    print('epoch', t, 'image generted')

# save images as a gif    
imageio.mimwrite('./curve_1.gif', my_images, fps=10)
plt.show()


In [ ]:
# show animated gif
print('Show training steps')
Image(open('curve_1.gif','rb').read())

# 코드 살펴보기

학습데이터 생성  
$y=x^{2}+\delta$  
$\delta$: random noise

-1과 1 사이에 동일한 간격으로 100개의 값을 추출하여 x값으로 사용  
torch.unsqeeze(..., dim=1) 함수로 1차원 텐서를 2차원 텐서로 변환  


In [ ]:
torch.manual_seed(1)    # 실행할 때마다 같은 결과를 내기 위해 random seed를 고정

x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)


In [ ]:
torch.linspace(-1, 1, 100)

In [ ]:
torch.linspace(-1, 1, 100).shape

In [ ]:
torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1).shape

In [ ]:
torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)

In [ ]:
x

In [ ]:
x.size()

In [ ]:
x.pow(2)

In [ ]:
torch.rand(x.size())

In [ ]:
y

In [ ]:

# view data
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis Training Data')
plt.xlabel('Independent varible (x)')
plt.ylabel('Dependent varible (y)')
plt.show()


In [ ]:
# simple two layer MLP. 1 -> 10 -> 1
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(n_feature=1, n_hidden=10, n_output=1)     # define the network
print(net)  # net architecture


Stochastic Gradient Descent optimizer  
Loss는 Mean Square Error  
$loss = \frac{1}{n}{\sum(\hat{y}_{i} - y_{i})^{2}}$

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

MSELoss 동작 확인

2x2 입력과 2x2 target 생성

In [ ]:
loss_test = torch.nn.MSELoss()
input_test = torch.randn(2, 2, requires_grad=True)
target_test = torch.randn(2, 2)


In [ ]:
input_test

In [ ]:
target_test

In [ ]:
output_test = loss_test(input_test, target_test)

In [ ]:
output_test

MSE 결과 비교

autograd 텐서는 numpy로 변환하기 전에 detach()를 해야 한다

In [ ]:
inp = input_test.detach().numpy()
tgt = target_test.detach().numpy()

In [ ]:
inp.shape

In [ ]:
inp

In [ ]:
tgt

In [ ]:
error_sum = 0
for i in range(inp.shape[0]):
    for j in range(inp.shape[1]):
        print('input_test', i, j, '= ', inp[i,j])
        print('target_test', i, j, '= ', tgt[i,j])
        error_squared = ( inp[i,j] - tgt[i,j])**2
        print('squared error = ', error_squared)
        error_sum += error_squared
print('Sum', error_sum)
print('Loss', error_sum / (inp.shape[0] * inp.shape[1]))

In [ ]:
output_test.backward()

In [ ]:
output_test

학습을 진행하며 중간결과는 plotting을 위해 저장

In [ ]:
my_images = []
fig, ax = plt.subplots(figsize=figsize)

# train the network for 200 epochs
for t in range(200):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # network output과 target 값으로 loss 계산

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    # plot and show learning process
    plt.cla()
    ax.set_title('Regression Analysis', fontsize=24)
    ax.set_xlabel('Independent variable (x)', fontsize=18)
    ax.set_ylabel('Dependent variable (y)', fontsize=18)
    ax.set_xlim(-1.05, 1.5)
    ax.set_ylim(-0.25, 1.25)
    ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
    ax.plot(x.data.numpy(), prediction.data.numpy(), 'g-', lw=3)
    ax.text(1.0, 0.1, 'Step = %d' % t, fontdict={'size': 18, 'color':  'red'})
    ax.text(1.0, 0, 'Loss = %.4f' % loss.data.numpy(),
            fontdict={'size': 18, 'color':  'red'})

    # Used to return the plot as an image array 
    # (https://ndres.me/post/matplotlib-animated-gifs-easily/)
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    my_images.append(image)

이미지를 animated gif로 저장하여 단계별 변화 시각화

In [ ]:
# save images as a gif    
imageio.mimwrite('./curve_1.gif', my_images, fps=10)
plt.show()
# show animated gif
print('Show training steps')
Image(open('curve_1.gif','rb').read())

# Try another example - sine 함수

In [ ]:

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt

import numpy as np
import imageio

torch.manual_seed(1)    # reproducible

x = torch.unsqueeze(torch.linspace(-10, 10, 1000), dim=1)  # x data (tensor), shape=(100, 1)
y = torch.sin(x) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# torch can only train on Variable, so convert them to Variable
#x, y = Variable(x), Variable(y)
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "blue")
plt.title('Regression Analysis')
plt.xlabel('Independent varible')
plt.ylabel('Dependent varible')
plt.savefig('curve_2.png')
plt.show()

# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(1, 200),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 1),
    )

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

BATCH_SIZE = 64
EPOCH = 200

torch_dataset = Data.TensorDataset(x, y)

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

my_images = []
fig, ax = plt.subplots(figsize=(16,10))

# start training
for epoch in range(EPOCH):
    print('epoch', epoch)
    for step, (batch_x, batch_y) in enumerate(loader): # for each training step
        
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = net(b_x)     # input x and predict based on x

        loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)

        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

        if step == 1:
            # plot and show learning process
            plt.cla()
            ax.set_title('Regression Analysis - model 3 Batches', fontsize=35)
            ax.set_xlabel('Independent variable', fontsize=24)
            ax.set_ylabel('Dependent variable', fontsize=24)
            ax.set_xlim(-11.0, 13.0)
            ax.set_ylim(-1.1, 1.2)
            ax.scatter(b_x.data.numpy(), b_y.data.numpy(), color = "blue", alpha=0.2)
            ax.scatter(b_x.data.numpy(), prediction.data.numpy(), color='green', alpha=0.5)
            ax.text(8.8, -0.8, 'Epoch = %d' % epoch,
                    fontdict={'size': 24, 'color':  'red'})
            ax.text(8.8, -0.95, 'Loss = %.4f' % loss.data.numpy(),
                    fontdict={'size': 24, 'color':  'red'})

            # Used to return the plot as an image array 
            # (https://ndres.me/post/matplotlib-animated-gifs-easily/)
            fig.canvas.draw()       # draw the canvas, cache the renderer
            image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

            my_images.append(image)

    


단계별 상태

In [ ]:
# save images as a gif    
imageio.mimsave('./curve_2_model_3_batch.gif', my_images, fps=4)

Image(open('curve_2_model_3_batch.gif','rb').read())


최종 결과 플로팅

In [ ]:

fig, ax = plt.subplots(figsize=(16,10))
plt.cla()
ax.set_title('Regression Analysis - model 3, Batches', fontsize=35)
ax.set_xlabel('Independent variable', fontsize=24)
ax.set_ylabel('Dependent variable', fontsize=24)
ax.set_xlim(-11.0, 13.0)
ax.set_ylim(-1.1, 1.2)
ax.scatter(x.data.numpy(), y.data.numpy(), color = "blue", alpha=0.2)
prediction = net(x)     # input x and predict based on x
ax.scatter(x.data.numpy(), prediction.data.numpy(), color='green', alpha=0.5)
plt.savefig('curve_2_model_3_batches.png')
plt.show()